## 协议简介
协议协调一系列计划，将其部署在远距离的工作人员上，并一次通过。

它是一个高级对象，其中包含分布在多个工作程序中的复杂计算的逻辑。协议的主要特征是能够在工作人员之间发送/搜索/取回，并最终部署到已确定的工作人员的能力。因此，用户可以设计协议，然后将其上载到Cloud Worker，其他任何Worker都可以搜索，下载并在其所连接的Worker上应用其包含的计算程序。

## 1 创建部署
通过提供成对列表来创建协议(worker, plan)。worker可以是实际工作人员，也可以是工作人员ID，也可以是表示虚拟工作人员的字符串。在创建时可以使用后一种情况来指定部署时同一工作人员应拥有（或不拥有）两个计划。plan可以是计划或PointerPlan。

In [1]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False

In [2]:
# 定义一系列计划
@sy.func2plan(args_shape=[(1,)])
def inc1(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc2(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc3(x):
    return x + 1

protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])

In [4]:
# 将协议绑定到worker
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")

workers = alice, bob, charlie
protocol.deploy(*workers)

<Protocol id:42453897586 owner:me resolved>
 - alice: [PointerPlan | me:67767771677 -> alice:59400908940]
 - bob: [PointerPlan | me:73508574980 -> bob:39724673319]
 - charlie: [PointerPlan | me:21217029022 -> charlie:68100273012]

## 2. 运行协议
运行协议意味着依次执行所有计划。为此，您提供一些输入数据，该数据将发送到第一个计划位置。运行第一个计划，并将其输出移到第二个计划位置，依此类推。运行完所有计划后，将返回最终结果，该结果由指向最后一个计划位置的指针组成。

In [6]:
x = th.tensor([1.0])
ptr = protocol.run(x)
ptr

send alice
move alice  ->  bob
move bob  ->  charlie


(Wrapper)>[PointerTensor | me:18000403678 -> charlie:54564292641]

In [7]:
# 输入1.0经历了3个计划，因此增加了3倍，这就是为什么它现在等于4.0的原因！
ptr.get()

tensor([4.])

In [8]:
# 运行远程协议。protocol也能发送到远程。
james = sy.VirtualWorker(hook, id="james")
protocol.send(james)

In [9]:
x = th.tensor([1.0]).send(james)
ptr = protocol.run(x)
ptr

send remote run request to james
send alice
move alice  ->  bob
move bob  ->  charlie


(Wrapper)>[PointerTensor | me:53893386564 -> james:23188330447]

In [10]:
ptr = ptr.get()
ptr

(Wrapper)>[PointerTensor | me:23188330447 -> charlie:73651850441]

In [11]:
ptr = ptr.get()
ptr

tensor([4.])

## 3. 搜索协议
在实际设置中，您可能希望下载一个远程协议，将其部署在您的工作程序上并与您的数据一起运行：

In [12]:
protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])
protocol.tag('my_protocol')
protocol.send(james)

In [13]:
# 得到当前的worker
me = sy.hook.local_worker # get access to me as a local worker

In [14]:
# 在james上搜索协议
responses = me.request_search(['my_protocol'], location=james)
responses

[[PointerProtocol | me:125920778 -> james:69669625366]]

In [15]:
# 得到第一条协议
ptr_protocol = responses[0]

In [16]:
# 去除协议中的计划
protocol_back = ptr_protocol.get()
protocol_back

<Protocol id:69669625366 owner:me>
 - me: <Plan inc1 id:59400908940 owner:me built>
def inc1(arg_1):
    _2 = _1.__add__(1)
    return _2
 - me: <Plan inc2 id:39724673319 owner:me built>
def inc2(arg_1):
    _2 = _1.__add__(1)
    return _2
 - me: <Plan inc3 id:68100273012 owner:me built>
def inc3(arg_1):
    _2 = _1.__add__(1)
    return _2

In [17]:
# 运行协议
protocol_back.deploy(alice, bob, charlie)

x = th.tensor([1.0])
ptr = protocol_back.run(x)
ptr.get()

send alice
move alice  ->  bob
move bob  ->  charlie


tensor([4.])